# Simple dose computation and optimization on a real CT image

In [2]:
import os

import numpy as np

from matplotlib import pyplot as plt

from opentps.core.data.images import CTImage
from opentps.core.data.images import ROIMask
from opentps.core.data.plan import PlanDesign
from opentps.core.data import DVH
from opentps.core.data import Patient
from opentps.core.io import mcsquareIO
from opentps.core.io.scannerReader import readScanner
from opentps.core.processing.doseCalculation.doseCalculationConfig import DoseCalculationConfig
from opentps.core.processing.doseCalculation.mcsquareDoseCalculator import MCsquareDoseCalculator
from opentps.core.processing.imageProcessing.resampler3D import resampleImage3DOnImage3D
from opentps.core.io.dataLoader import readData
from opentps.core.data.plan import ObjectivesList
from opentps.core.data.plan import FidObjective
from opentps.core.io.dicomIO import readDicomPlan, readDicomStruct
from opentps.core.data._rtStruct import RTStruct
from opentps.core.io.serializedObjectIO import saveBeamlets, saveRTPlan, loadBeamlets, loadRTPlan


In [3]:
ctCalibration = readScanner(DoseCalculationConfig().scannerFolder)
bdl = mcsquareIO.readBDL(DoseCalculationConfig().bdlFile)

In [4]:
ctImagePath = "Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain"
data = readData(ctImagePath)

Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain/CT/1.2.826.0.1.3680043.6.11113.80068.20170905183644.6880.5.172.dcm
Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain
Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain/CT/1.2.826.0.1.3680043.6.11219.36586.20170905183631.6880.5.19.dcm
Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain
Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain/CT/1.2.826.0.1.3680043.6.11893.99074.20170905183643.6880.5.157.dcm
Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain
Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain/CT/1.2.826.0.1.3680043.6.12173.3462.20170905183639.6880.5.122.dcm
Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain
Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain/CT/1.2.826.0.1.3680043.6.1234.42591.20170905183641.6880.5.145.dcm
Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain
Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain/CT/1.2.826.0.1.3680043.6.12874.61703.20170905183636.6880.5.91.dcm
Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain
Data/ProKnows_2018_TROG_Plan_Study_SRS_Brain/CT/1.2.826.0.1.36

In [5]:
rt_struct = data[0]
ct = data[1]

In [6]:
print(rt_struct.print_ROINames())


RT Struct UID: 2.16.840.1.114362.1.6.6.9.161209.11109530676.470491132.964.204
  [0]  GTV1-20Gy
  [1]  GTV2-20Gy
  [2]  GTV3-20Gy
  [3]  GTV4-20Gy
  [4]  Eye_L
  [5]  Eye_R
  [6]  Lens_L
  [7]  Lens_R
  [8]  OpticNerve_L
  [9]  OpticNerve_R
  [10]  Optic Chiasm
  [11]  Hippocampus_L
  [12]  Hippocampus_R
  [13]  Brainstem
  [14]  Brain
  [15]  BODY
  [16]  Bones
  [17]  Normal Brain
  [18]  GTV5-20Gy
  [19]  GTV-Total
None


In [7]:
target_name = "GTV4-20Gy"
target = rt_struct.getContourByName(target_name).getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing)

OAR_brain = rt_struct.getContourByName("Brain").getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing)
OAR_brainstem = rt_struct.getContourByName("Brainstem").getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing)
OAR_optic_chiasm = rt_struct.getContourByName("Optic Chiasm").getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing)

In [8]:
COM_coord = target.centerOfMass
COM_index = target.getVoxelIndexFromPosition(COM_coord)
Z_COORD = COM_index[2]

In [9]:
# Configure MCsquare
mc2 = MCsquareDoseCalculator()
mc2.beamModel = bdl
mc2.ctCalibration = ctCalibration

In [26]:
# Design plan
beamNames = ["Beam1","Beam2","Beam3"]
gantryAngles = [0.,90.,270.]
couchAngles = [0.,0.,0.]

# Generate new plan
planDesign = PlanDesign()
planDesign.ct = ct
planDesign.gantryAngles = gantryAngles
planDesign.targetMask = target
planDesign.beamNames = beamNames
planDesign.couchAngles = couchAngles
planDesign.calibration = ctCalibration
planDesign.spotSpacing = 5.0
planDesign.layerSpacing = 5.0
planDesign.targetMargin = 5.0

plan = planDesign.buildPlan()  # Spot placement
plan.PlanName = "NewPlan"

07/08/2023 11:27:54 AM - opentps.core.data.plan._planDesign - INFO - Building plan ...
07/08/2023 11:27:56 AM - opentps.core.processing.planOptimization.planInitializer - INFO - Target is dilated using a margin of 5.0 mm. This process might take some time.
07/08/2023 11:27:56 AM - opentps.core.processing.imageProcessing.roiMasksProcessing - INFO - Using SITK to dilate mask.
07/08/2023 11:28:39 AM - opentps.core.data.plan._planDesign - INFO - New plan created in 44.996827125549316 sec
07/08/2023 11:28:39 AM - opentps.core.data.plan._planDesign - INFO - Number of spots: 629


In [27]:
mc2.nbPrimaries = 5e4
beamlets = mc2.computeBeamlets(ct, plan)
plan.planDesign.beamlets = beamlets

07/08/2023 11:29:02 AM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Prepare MCsquare Beamlet calculation
07/08/2023 11:29:06 AM - opentps.core.io.mhdIO - INFO - Write MHD file: /home/eliot/openTPS_workspace/Simulations/MCsquare_simulation/CT.mhd
07/08/2023 11:29:06 AM - opentps.core.io.mcsquareIO - INFO - Write plan: /home/eliot/openTPS_workspace/Simulations/MCsquare_simulation/PlanPencil.txt
07/08/2023 11:29:07 AM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Start MCsquare simulation


MCsquare: 7: [: Linux: unexpected operator





Initialization time: 0.799768 s 


Simulation of beamlet 201/629  
MC computation time: 4.577083 s 
Output computation time: 0.911998 s 

Simulation of beamlet 190/629  
MC computation time: 4.980151 s 
Output computation time: 0.897979 s 

Simulation of beamlet 157/629  
MC computation time: 5.012774 s 
Output computation time: 0.964779 s 

Simulation of beamlet 168/629  
MC computation time: 5.178339 s 
Output computation time: 0.966095 s 

Simulation of beamlet 145/629  
MC computation time: 5.393389 s 
Output computation time: 0.895934 s 

Simulation of beamlet 133/629  
MC computation time: 5.511328 s 
Output computation time: 0.910991 s 

Simulation of beamlet 179/629  
MC computation time: 5.715948 s 
Output computation time: 0.917684 s 

Simulation of beamlet 109/629  
MC computation time: 5.928505 s 
Output computation time: 0.965087 s 

Simulation of beamlet 121/629  
MC computation time: 5.981175 s 
Output computation time: 0.950151 s 

Simulation of beamlet 85/629  
MC c

MCsquare: 7: [: Linux: unexpected operator





Initialization time: 0.797903 s 


Simulation started (2023-08-07 11:43:50) 
 10.0 % 
 20.0 % 
 30.0 % 
 40.0 % 
 50.0 % (stat uncertainty: 2.83 %) 
 60.0 % (stat uncertainty: 2.67 %) 
 70.0 % (stat uncertainty: 2.54 %) 
 80.0 % (stat uncertainty: 2.41 %) 
 90.0 % (stat uncertainty: 2.30 %) 
 100.0 % (stat uncertainty: 2.21 %) 

Nbr primaries simulated: 10000000 (6333 generated outside the geometry) 
MC computation time: 43.277157 s 
Output computation time: 0.174737 s 

Total computation time: 44.284607 s 


In [10]:
# Save the plan and the beamlets
#saveBeamlets(beamlets,"Data/SimpleRealDoseComputationOptimization_beamlets")
#saveRTPlan(plan,"Data/SimpleRealDoseComputationOptimization_plan.tps")
plan = loadRTPlan("Data/SimpleRealDoseComputationOptimization_plan.tps")
plan.planDesign.beamlets = loadBeamlets("Data/SimpleRealDoseComputationOptimization_beamlets")

In [11]:
mc2.nbPrimaries = 1e7
dose_before_opti = mc2.computeDose(ct,plan)

07/08/2023 02:03:29 PM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Prepare MCsquare Dose calculation
07/08/2023 02:03:29 PM - opentps.core.io.mhdIO - INFO - Write MHD file: /home/eliot/openTPS_workspace/Simulations/MCsquare_simulation/CT.mhd
07/08/2023 02:03:30 PM - opentps.core.io.mcsquareIO - INFO - Write plan: /home/eliot/openTPS_workspace/Simulations/MCsquare_simulation/PlanPencil.txt
07/08/2023 02:03:32 PM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Start MCsquare simulation


MCsquare: 7: [: Linux: unexpected operator





Initialization time: 1.215820 s 


Simulation started (2023-08-07 14:03:33) 
 10.0 % 
 20.0 % 
 30.0 % 
 40.0 % 
 50.0 % (stat uncertainty: 2.85 %) 
 60.0 % (stat uncertainty: 2.69 %) 
 70.0 % (stat uncertainty: 2.55 %) 
 80.0 % (stat uncertainty: 2.42 %) 
 90.0 % (stat uncertainty: 2.31 %) 
 100.0 % (stat uncertainty: 2.21 %) 

Nbr primaries simulated: 10000000 (6300 generated outside the geometry) 
MC computation time: 43.464983 s 
Output computation time: 0.175948 s 

Total computation time: 44.892974 s 


In [12]:
plan.planDesign.objectives = ObjectivesList() #create a new objective set
plan.planDesign.objectives.setTarget(target.name, 20.0) #setting a target of 20 Gy for the target
plan.planDesign.objectives.fidObjList = []
plan.planDesign.objectives.addFidObjective(target, FidObjective.Metrics.DMAX, 20.0, 1.0)
plan.planDesign.objectives.addFidObjective(target, FidObjective.Metrics.DMIN, 20.0, 1.0)
plan.planDesign.defineTargetMaskAndPrescription()

In [15]:
from opentps.core.processing.planOptimization.planOptimization import IMPTPlanOptimizer

solver = IMPTPlanOptimizer(method='Scipy-LBFGS',plan=plan,maxit=100)
w, doseImage, ps = solver.optimize()

plan.spotMUs = np.square(w).astype(np.float32)

07/08/2023 02:05:01 PM - opentps.core.processing.planOptimization.planOptimization - INFO - Prepare optimization ...


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          551     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.35664D+02    |proj g|=  3.01264D+00
07/08/2023 02:05:09 PM - opentps.core.processing.planOptimization.solvers.bfgs - INFO - Iteration 1 of Scipy-L-BFGS-B

At iterate    1    f=  2.92808D+01    |proj g|=  2.22289D+00
07/08/2023 02:05:09 PM - opentps.core.processing.planOptimization.solvers.bfgs - INFO - objective = 2.928081e+01  
07/08/2023 02:05:12 PM - opentps.core.processing.planOptimization.solvers.bfgs - INFO - Iteration 2 of Scipy-L-BFGS-B

At iterate    2    f=  2.52496D+00    |proj g|=  3.63914D-01
07/08/2023 02:05:12 PM - opentps.core.processing.planOptimization.solvers.bfgs - INFO - objective = 2.524961e+00  
07/08/2023 02:05:13 PM - opentps.core.processing.planOptimization.solvers.bfgs - INFO - Iteration 3 of Scipy-L-BFGS-B

At iterate    3    f=  2.04052D+00    |proj g|=  2.30879D-01
0

In [14]:
print(len(w))
print(plan.numberOfSpots)

551
551


In [14]:
finalDoseImage = mc2.computeDose(ct,plan)

28/07/2023 12:40:17 PM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Prepare MCsquare Dose calculation
28/07/2023 12:40:17 PM - opentps.core.io.mcsquareIO - INFO - Cropping CT around External
28/07/2023 12:40:18 PM - opentps.core.io.mhdIO - INFO - Write MHD file: /home/eliot/openTPS_workspace/Simulations/MCsquare_simulation/CT.mhd
28/07/2023 12:40:18 PM - opentps.core.io.mcsquareIO - INFO - Write plan: /home/eliot/openTPS_workspace/Simulations/MCsquare_simulation/PlanPencil.txt
28/07/2023 12:40:20 PM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Start MCsquare simulation


MCsquare: 7: [: Linux: unexpected operator





Initialization time: 0.731086 s 


Simulation started (2023-07-28 12:40:21) 
 10.0 % 
 20.0 % 
 30.0 % 
 40.0 % 
 50.0 % (stat uncertainty: 27.00 %) 
 60.0 % (stat uncertainty: 25.60 %) 
 70.0 % (stat uncertainty: 24.45 %) 
 80.0 % (stat uncertainty: 23.28 %) 
 90.0 % (stat uncertainty: 22.30 %) 
 100.0 % (stat uncertainty: 21.61 %) 

Nbr primaries simulated: 50080 
MC computation time: 12.645139 s 
Output computation time: 0.132509 s 

Total computation time: 13.533063 s 
